In [1]:
import pandas as pd

data = pd.read_csv('ionosphere.csv')

data.head()

,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.1,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
0,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
1,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
2,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
3,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g
4,1,0,0.02337,-0.00592,-0.09924,-0.11949,-0.00763,-0.11824,0.14706,0.06637,...,-0.01535,-0.03240,0.09223,-0.07859,0.00732,0.00000,0.00000,-0.00039,0.12011,b


In [2]:
data['g'].unique()

array(['b', 'g'], dtype=object)

In [3]:
import torch
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader, random_split
from torch import Tensor, device, cuda
from torch.nn import Linear, ReLU, Sigmoid, Module, BCELoss
from torch.optim import SGD
from torch.nn.init import kaiming_uniform_, xavier_uniform_

# Detectar si hay GPU disponible
device_type = device("cuda" if cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device_type}")

# Definición del dataset
class CSVDataset(Dataset):
    def __init__(self, path):
        # Cargar dataset desde el archivo
        df = read_csv(path, header=None)
        self.X = df.values[:, :-1]  # Todas las columnas menos la última (características)
        self.y = df.values[:, -1]  # Última columna (etiquetas)
        self.X = self.X.astype('float32')
        # Convertir las etiquetas en valores binarios (0 o 1)
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('float32').reshape((len(self.y), 1))

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    def get_splits(self, n_test=0.33):
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        return random_split(self, [train_size, test_size])

def prepare_data(path):
    dataset = CSVDataset(path)
    train, test = dataset.get_splits()  # Dividir en entrenamiento y prueba
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

# Definición del modelo MLP
class MLP(Module):
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # Primera capa oculta
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # Segunda capa oculta
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # Capa de salida
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()

    def forward(self, X):
        X = self.hidden1(X)
        X = self.act1(X)
        X = self.hidden2(X)
        X = self.act2(X)
        X = self.hidden3(X)
        X = self.act3(X)
        return X

# Entrenar el modelo
def train_model(train_dl, test_dl, model, epochs=50):
    criterion = BCELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    model.to(device_type)

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_dl:
            inputs, targets = inputs.to(device_type), targets.to(device_type)
            optimizer.zero_grad()
            yhat = model(inputs)
            loss = criterion(yhat, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validación
        model.eval()
        val_loss = 0.0
        predictions, actuals = list(), list()
        with torch.no_grad():
            for inputs, targets in test_dl:
                inputs, targets = inputs.to(device_type), targets.to(device_type)
                yhat = model(inputs)
                loss = criterion(yhat, targets)
                val_loss += loss.item()
                yhat = yhat.cpu().numpy()
                actual = targets.cpu().numpy()
                yhat = yhat.round()
                predictions.append(yhat)
                actuals.append(actual)

        predictions, actuals = vstack(predictions), vstack(actuals)
        val_acc = accuracy_score(actuals, predictions)

        print(f"Epoch {epoch + 1}/{epochs}, "
              f"Train Loss: {train_loss / len(train_dl):.4f}, "
              f"Val Loss: {val_loss / len(test_dl):.4f}, "
              f"Val Accuracy: {val_acc:.4f}")

# Evaluar el modelo
def evaluate_model(test_dl, model):
    model.eval()
    predictions, actuals = list(), list()
    with torch.no_grad():
        for inputs, targets in test_dl:
            inputs, targets = inputs.to(device_type), targets.to(device_type)
            yhat = model(inputs)
            yhat = yhat.cpu().numpy()
            actual = targets.cpu().numpy()
            yhat = yhat.round()
            predictions.append(yhat)
            actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    acc = accuracy_score(actuals, predictions)
    return acc

# Hacer una predicción
def predict(row, model):
    row = Tensor([row]).to(device_type)
    model.eval()
    with torch.no_grad():
        yhat = model(row)
    return yhat.cpu().numpy()

# Cargar el dataset y dividirlo
path = 'ionosphere.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))  # Tamaño de los conjuntos

# Crear el modelo
model = MLP(34)

# Entrenar el modelo
train_model(train_dl, test_dl, model, epochs=50)

# Evaluar el modelo
acc = evaluate_model(test_dl, model)
print(f"Precisión final en el conjunto de prueba: {acc:.3f}")


Usando dispositivo: cuda
235 116
Epoch 1/50, Train Loss: 0.7280, Val Loss: 0.6892, Val Accuracy: 0.6207
Epoch 2/50, Train Loss: 0.6719, Val Loss: 0.6535, Val Accuracy: 0.6379
Epoch 3/50, Train Loss: 0.6347, Val Loss: 0.6302, Val Accuracy: 0.6121
Epoch 4/50, Train Loss: 0.6207, Val Loss: 0.6074, Val Accuracy: 0.6207
Epoch 5/50, Train Loss: 0.5785, Val Loss: 0.5867, Val Accuracy: 0.6552
Epoch 6/50, Train Loss: 0.5758, Val Loss: 0.5675, Val Accuracy: 0.6810
Epoch 7/50, Train Loss: 0.5491, Val Loss: 0.5492, Val Accuracy: 0.7328
Epoch 8/50, Train Loss: 0.5301, Val Loss: 0.5287, Val Accuracy: 0.7586
Epoch 9/50, Train Loss: 0.5270, Val Loss: 0.5057, Val Accuracy: 0.7931
Epoch 10/50, Train Loss: 0.5079, Val Loss: 0.4843, Val Accuracy: 0.8017
Epoch 11/50, Train Loss: 0.4893, Val Loss: 0.4616, Val Accuracy: 0.8276
Epoch 12/50, Train Loss: 0.4464, Val Loss: 0.4407, Val Accuracy: 0.8276
Epoch 13/50, Train Loss: 0.4355, Val Loss: 0.4199, Val Accuracy: 0.8276
Epoch 14/50, Train Loss: 0.4409, Val Los

In [5]:
def predict(row, model):
    # Convertimos la fila en un tensor y lo movemos al dispositivo adecuado
    row = Tensor([row]).to(device_type)
    
    # Ponemos el modelo en modo de evaluación
    model.eval()
    
    # Realizamos la predicción sin calcular gradientes
    with torch.no_grad():
        yhat = model(row)
    
    # Devolvemos la predicción como un array de NumPy
    return yhat.cpu().numpy()

# Definimos una nueva fila de datos para hacer la predicción
row = [1, 0, 0.99539, -0.05889, 0.85243, 0.02306, 0.83398, -0.37708, 1, 0.03760, 
       0.85243, -0.17755, 0.59755, -0.44945, 0.60536, -0.38223, 0.84356, -0.38542, 
       0.58212, -0.32192, 0.56971, -0.29674, 0.36946, -0.47357, 0.56811, -0.51171, 
       0.41078, -0.46168, 0.21266, -0.34090, 0.42267, -0.54487, 0.18641, -0.45300]

# Realizamos la predicción
yhat = predict(row, model)

# Mostramos la probabilidad predicha y la clase (0 o 1)
print(f'Probabilidad predicha: {yhat[0][0]:.3f}, Clase: {int(yhat[0][0].round())}')


Probabilidad predicha: 0.938, Clase: 1
